# Exercise 1:
Categorical Variables

In [10]:
import pandas as pd
df = pd.DataFrame({'salary': [103, 89, 142, 54, 63, 219],
                   'boro': ['Manhattan', 'Queens', 'Manhattan', 'Brooklyn', 'Brooklyn', 'Bronx']})
df

,salary,boro
0,103,Manhattan
1,89,Queens
2,142,Manhattan
3,54,Brooklyn
4,63,Brooklyn
5,219,Bronx


In [11]:
df.dtypes=='object'

salary    False
boro       True
dtype: bool

In [12]:
pd.get_dummies(df,dtype=int)

,salary,boro_Bronx,boro_Brooklyn,boro_Manhattan,boro_Queens
0,103,0,0,1,0
1,89,0,0,0,1
2,142,0,0,1,0
3,54,0,1,0,0
4,63,0,1,0,0
5,219,1,0,0,0


In [13]:
from sklearn.preprocessing import OneHotEncoder

In [14]:
ohe1 = OneHotEncoder(sparse_output=False)

In [15]:
ohe1.fit(df)

OneHotEncoder(sparse_output=False)

In [16]:
ohe1.transform(df)

array([[0., 0., 0., 1., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 1., 0., 0., 0.]])

In [17]:
ohe1

OneHotEncoder(sparse_output=False)

In [18]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler

In [19]:
df.dtypes

salary     int64
boro      object
dtype: object

In [20]:
categorical = df.dtypes == 'object'

In [21]:
~categorical

salary     True
boro      False
dtype: bool

In [22]:
tr1 = make_column_transformer((StandardScaler(), ~categorical), (OneHotEncoder(), categorical))

In [23]:
tr1

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 salary     True
boro      False
dtype: bool),
                                ('onehotencoder', OneHotEncoder(),
                                 salary    False
boro       True
dtype: bool)])

In [24]:
tr1.transform(df)

NotFittedError: This ColumnTransformer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
StandardScaler().fit_transform(df[['salary']])

In [ ]:
tr1.get_feature_names_out()

In [31]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


df = pd.DataFrame({'brand': ['BMW', 'Pepsi', 'Mac', np.nan],
                   'category': ['car', 'drink', 'computer', 'Unknown'],
                   'num1': [1, 1, 0, 0],
                   'target': [0.2, 0.11, 1.34, 1.123]})

In [32]:
df

,brand,category,num1,target
0,BMW,car,1,0.200
1,Pepsi,drink,1,0.110
2,Mac,computer,0,1.340
3,NaN,Unknown,0,1.123


In [33]:
df.dtypes==object

brand        True
category     True
num1        False
target      False
dtype: bool

In [29]:
num = ['num1']

In [30]:
cat = ['brand', 'category']

In [34]:
df.isna()

,brand,category,num1,target
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,True,False,False,False


In [35]:
num_tr = Pipeline(steps = [('impnum', SimpleImputer(strategy='median')),('scaler', StandardScaler())])

In [36]:
num_tr

Pipeline(steps=[('impnum', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [40]:
num_tr.steps

[('impnum', SimpleImputer(strategy='median')), ('scaler', StandardScaler())]

In [41]:
cat_tr = Pipeline(steps = [('impcat', SimpleImputer(strategy='constant', fill_value='missing')),('onehot', OneHotEncoder())])

In [42]:
tr2 = ColumnTransformer(transformers = [('num', num_tr, num), ('cat', cat_tr, cat)])

In [45]:
tr2

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('impnum',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['num1']),
                                ('cat',
                                 Pipeline(steps=[('impcat',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot', OneHotEncoder())]),
                                 ['brand', 'category'])])

In [43]:
tr2.fit_transform(df)

array([[ 1.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.],
       [-1.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.],
       [-1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.]])

In [44]:
tr2.get_feature_names_out()

array(['num__num1', 'cat__brand_BMW', 'cat__brand_Mac',
       'cat__brand_Pepsi', 'cat__brand_missing', 'cat__category_Unknown',
       'cat__category_car', 'cat__category_computer',
       'cat__category_drink'], dtype=object)

In [46]:
X = df.drop('target', axis=1)
y = df['target']

In [47]:
from sklearn.neighbors import KNeighborsRegressor

In [50]:
reg = Pipeline(steps=[('prep', tr2), ('knr', KNeighborsRegressor(n_neighbors=2))])

In [51]:
reg

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('impnum',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['num1']),
                                                 ('cat',
                                                  Pipeline(steps=[('impcat',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['brand', 'category'])])),
                ('knr', KNeighborsRegressor(n_neighbors=2))])

In [53]:
reg.fit(X, y)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('impnum',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['num1']),
                                                 ('cat',
                                                  Pipeline(steps=[('impcat',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['brand', 'category'])])),
                ('knr', KNeighborsRegressor(n_neighbors=2))])

In [54]:
reg.score(X, y)

0.9767418976030741

In [55]:
from sklearn.model_selection import GridSearchCV

In [63]:
# reg__knnr comes from the fact that we have a variable named reg
# and a variable inside reg called knnr
myparams = {'reg__knnr__n_neighbors':np.arange(1,3,1)}
mygrid = GridSearchCV(reg, param_grid=myparams, cv=1)

In [64]:
mygrid

GridSearchCV(cv=1,
             estimator=Pipeline(steps=[('prep',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('impnum',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['num1']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('impcat',
                                                                                          SimpleImputer(fill_value='missing',
                                                                                                        strategy='constant')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder())]),
                                                                         ['brand',
                                                                          'category'])])),
                                       ('knr',
                                        KNeighborsRegressor(n_neighbors=2))]),
             param_grid={'reg__knnr__n_neighbors': array([1, 2])})

In [ ]:
mygrid.fit(X, y)